In [2]:
import pandas as pd
import os
import shutil
import glob

### Получение результатов модели

**Из корня проекта нужно запустить run_models скрипт:**
```console
    python3 run_models.py
```

* Новые таблички сохранятся в директорию модели, с помощью которой эта таблица была получена.
* Модель и данные можно поменять в configs/mining_config.yaml

In [12]:
def dockerpath_to_real(name: str) -> str:
    return os.path.join('../../', '/'.join(name.split('/')[3:]))

In [13]:
df_path = '../../outputs/stochastic_depth/version.Model.architecture.stochastic_depth_p=0.0,version=stochastic_depth_comparation/mining_sorce.csv'
results = pd.read_csv(df_path, index_col=0)
results.Filepath = results.Filepath.apply(lambda name: dockerpath_to_real(name))
results.head()

,Filepath,Sex_pr,Age_pr,Skin_Color_pr,Headwear_pr,Hair_color_pr,Glasses_pr,Scarf_pr,Backpack_pr,Main_Color_pr,...,Sex_gt,Age_gt,Skin_Color_gt,Headwear_gt,Hair_color_gt,Glasses_gt,Scarf_gt,Backpack_gt,Main_Color_gt,UpperBodyWearType_gt
0,../../data/Datasets/Metro/MiningPart1UB/images...,1,1,0,1,4,2,1,1,7,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,../../data/Datasets/Metro/MiningPart1UB/images...,1,2,0,1,1,1,1,1,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,../../data/Datasets/Metro/MiningPart1UB/images...,1,2,0,0,2,1,0,1,7,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,../../data/Datasets/Metro/MiningPart1UB/images...,0,1,0,1,2,1,1,1,7,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,../../data/Datasets/Metro/MiningPart1UB/images...,0,2,2,0,1,2,1,0,7,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


### Майнинг 

#### Самые уверенные

In [14]:
threshold = 0.8

##### Upperbody

In [32]:
def to_text_upper(val: int, attribute: str):
    if val == -1:
        return f"Непонятно"
    descr = {
        'Backpack': ['Есть', 'Нет'],
        'Scarf': ['Есть', 'Нет'],
        'Sex': ['Муж', 'Жен'],
        'Age': ['Ребенок', 'Молодой', 'Взрослый', 'Пожилой'],
        'Skin_Color': ['Белый', 'Черный', 'Не видно'],
        'Hair_color': ['Лысый', 'Блонд', 'Темные', 'Цветные', 'Не видно'],
        'Headwear': ['Есть', 'Нет'],
        'Glasses': ['Есть', 'Нет', 'Не видно'],
        'Main_Color': ['Белый', 'Красный', 'Желтый' , 'Зеленый', 'Синий', 'Серый', 'Коричневый', 'Черный'],
        'UpperBodyWearType': ['Футболка', 'Пиджак or Жакет', 'Куртка', 'Кофты or толстовки', 'Жилет', 'Рубашка'],
    }
    return f'{descr[attribute][val]}'

In [33]:
attributes_list = ["Sex", "Age", "Skin_Color", "Headwear", "Hair_color", "Glasses", "UpperBodyWearType", "Scarf", "Backpack", "Main_Color"]

# Path to save
base_path = f"outputs/UpperBody/Name"
if not os.path.exists(base_path):
    os.makedirs(base_path)

for attribute in attributes_list:
    values = set(results[f"{attribute}_pr"].tolist())
    dir_path = f"{base_path}/{attribute}"
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
        
    for val in values:
        dir_path = f"{base_path}/{attribute}/{to_text_upper(val, attribute)}"
        if not os.path.exists(dir_path):
            print(dir_path)
            os.makedirs(dir_path)
        files = glob.glob(f"{base_path}/{attribute}/{to_text_upper(val, attribute)}/*.jpg")
        for f in files:
            os.remove(f)
        ranked = results[results[f"{attribute}_pr"] == val]
        ranked = ranked[ranked[f"{attribute}_conf"] > threshold]
        for path, conf in zip(ranked.Filepath.tolist(), ranked[f"{attribute}_conf"].tolist()):
            shutil.copy(path, f"{base_path}/{attribute}/{to_text_upper(val, attribute)}")
            old_name = path.split("/")[-1]
            new_name = f"conf_{int(conf * 100)}_name_{old_name}"
            shutil.move(f"{base_path}/{attribute}/{to_text_upper(val, attribute)}/{old_name}", 
                        f"{base_path}/{attribute}/{to_text_upper(val, attribute)}/{new_name}")

outputs/UpperBody/Name/Main_Color/Белый
outputs/UpperBody/Name/Main_Color/Красный
outputs/UpperBody/Name/Main_Color/Желтый
outputs/UpperBody/Name/Main_Color/Зеленый
outputs/UpperBody/Name/Main_Color/Синий
outputs/UpperBody/Name/Main_Color/Серый
outputs/UpperBody/Name/Main_Color/Коричневый
outputs/UpperBody/Name/Main_Color/Черный


Также мы иногда майнили не самые уверенные, а примеры с уверенностью 0.4-0.8. 

В таком случае просто можно поменять 23 строчку предыдущей ячейки

#### FullBody

In [35]:
def to_text(val: int, attribute: str):
    if val == -1:
        return f"{type_}: Непонятно"
    descr = {
        'Bag': ['hand_bag', 'shoulder_bag', 'suitcase', 'nothing'],
        'Backpack': ['Yes', 'No']
    }
    return f'{descr[attribute][val]}'

In [ ]:
attributes_list = ["Bag", "Backpack"]

base_path = f"outputs/FullBody/Name"
if not os.path.exists(base_path):
    os.makedirs(base_path)

stats = dict()
for attribute in attributes_list:
    stats[attribute] = dict()
    values = set(results[f"{attribute}_pr"].tolist())
    dir_path = f"{base_path}/{attribute}"
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
        
    for val in values:
        dir_path = f"{base_path}/{attribute}/{to_text(val, attribute)}"
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)
        files = glob.glob(f"{base_path}/{attribute}/{to_text(val, attribute)}/*.jpg")
        for f in files:
            os.remove(f)
        ranked = results[results[f"{attribute}_pr"] == val]
        ranked = ranked[ranked[f"{attribute}_conf"] > threshold]
        stats[attribute][val] = len(ranked.filepath.tolist())
        for path in ranked.filepath.tolist():
            auto_ann[path.split("/")[-1]][attribute] = to_text(val, attribute)
            shutil.copy(path, f"{base_path}/{attribute}/{to_text(val, attribute)}")